<b>Задача:</b><br> 
выполнить кластеризацию кварталов Васильевского острова на основании предоставленных данных о границах кварталов и 
сервисах разных видов, расположенных на территории кварталов; также необходимо обосновать конечный выбор количества кластеров
и интерпретировать результаты кластеризац<br><br>
<b>Исходные данные:</b><
в качестве исходных данных из Платформы городских данных Института дизайна и урбанистики ИТМО выгружены следующие массивы данных:
- blocks.geojson - данные о кварталах Васильевского острова;
- cafe.geojson - данные о местах общественного питания типа кафе;
- restaurant.geojson - данные о местах общественного питания типа ресторан;
- kindergarten.geojson - данные о детских садах;
- school.geojson - данные о школах;
- library.geojson - данные о библиотеках;
- museum.geojson - данные о музеях;
- pharmacy.geojson - данные об аптеках;
- polyclinic.geojson - данные о поликлинниках;
- shopping_center.geojson - данные о торговых центрах;
- supermarket.geojson - данные о супермаркетах;
- building.geojson - данные о зданиях (жилых и нежилых);
- cemetery.geojson - данные о кладбищах;
- znop.geojson - данные о зелёных насаждениях.bх насаждениях.и. 

In [1]:
import pandas as pd
import geopandas as gpd

<b>1 Исследовательский анализ данных</b>

<b>1.1 Данные о кварталах</b>

In [2]:
blocks_data = gpd.read_file('data/source_data/blocks.geojson')
blocks_data.head()

,id,municipality,district,geometry
0,10,Остров Декабристов,Василеостровский,"MULTIPOLYGON (((30.24774 59.94989, 30.24657 59..."
1,16,округ Морской,Василеостровский,"MULTIPOLYGON (((30.21620 59.94924, 30.22653 59..."
2,46,Васильевский,Василеостровский,"MULTIPOLYGON (((30.26647 59.94468, 30.27092 59..."
3,67,округ Морской,Василеостровский,"MULTIPOLYGON (((30.20804 59.93856, 30.20854 59..."
4,112,Васильевский,Василеостровский,"MULTIPOLYGON (((30.28385 59.94769, 30.28505 59..."


In [3]:
blocks_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id            200 non-null    int64   
 1   municipality  200 non-null    object  
 2   district      200 non-null    object  
 3   geometry      200 non-null    geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 6.4+ KB


In [4]:
print('Представлены кварталы районов: ', blocks_data.district.unique())

Представлены кварталы районов:  ['Василеостровский']


В выборке содержатся данные 200 кварталов, пропусков в данных нет, геометрически кварталы представляют собой полигоны.
Признак district может быть удален (все кварталы относятся к Василеостровскому району), признак municipality также
следует удалить, так как кластеризация требуется на основании данных о сервисах. В итоге для дальнейшего анализа
будут использоваться данные о номерах кварталов (id) и их геометрии (geometry). 

In [5]:
blocks_data = blocks_data[['id', 'geometry']]

<b>1.2 Данные о местах общественного питания типа кафе</b>

In [6]:
cafe_data = gpd.read_file('data/source_data/cafe.geojson')
cafe_data.head()

,yand_adr,name,type,id,amenity,x,y,geometry
0,"Россия, Санкт-Петербург, Биржевой переулок, 2",Эдда,кафе,node/3168670772,cafe,59.945017,30.293563,POINT (30.29356 59.94502)
1,"Россия, Санкт-Петербург, 6-я линия Васильевско...",Ферма,кафе,node/6355459185,cafe,59.940343,30.282262,POINT (30.28226 59.94034)
2,"Россия, Санкт-Петербург, 1-я линия Васильевско...",Столовая,столовая,node/5235334623,cafe,59.943422,30.287877,POINT (30.28788 59.94342)
3,"Россия, Санкт-Петербург, 8-я линия Васильевско...",Рамен Шифу,кафе,node/963944161,cafe,59.937607,30.282020,POINT (30.28202 59.93761)
4,"Россия, Санкт-Петербург, улица Кораблестроител...",Небесный ФУД Город,кафе,node/825152812,cafe,59.952493,30.212346,POINT (30.21235 59.95249)


In [7]:
cafe_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   yand_adr  112 non-null    object  
 1   name      112 non-null    object  
 2   type      112 non-null    object  
 3   id        112 non-null    object  
 4   amenity   112 non-null    object  
 5   x         112 non-null    float64 
 6   y         112 non-null    float64 
 7   geometry  112 non-null    geometry
dtypes: float64(2), geometry(1), object(5)
memory usage: 7.1+ KB


In [8]:
print('Типы кафе: ', cafe_data['type'].unique())

Типы кафе:  ['кафе' 'столовая']


В выборке содержатся данные 112 кафе и столовых, пропусков в данных нет, геометрически сервисы этого типа представляют собой точки.
В итоге для дальнейшего анализа будут использоваться данные о номерах кафе (id) и их геометрии (geometry).

<b>1.3 Данные о прочих "точечных" сервисах</b>

Исследовательский анализ показал (эта часть не включается в ноутбук для сокращения его размера), что к сервисам с геометрией точки также 
относятся датасеты restaurant.geojson (70 сервисов), kindergarten.geojson (49 сервисов), school.geojson (39 сервисов),
library.geojson (13 сервисов), museum.geojson (26 сервисов), pharmacy.geojson (42 сервиса), polyclinic.geojson (8 сервисов) 
shopping_center.geojson (3 сервиса), supermarket.geojson (71 серви ).
Все перечисленные датасеты имски аналогичную структуру, не содержат пропусков, и для дальнейшего анализа буут 
использоваться данные id и geometry из этих датас<br>етов. 
Таким образом, типов точечных сервисов достаточно много, но количество сервисов каждого из типов достаточно ограничено. 
Причем есть типы сервисов, очень похожие с точки зрения потребителей услуг. Например, кафе и рестораны (места общественного питания)
или торговые центры и супермаркеты (места для покупок). Для сокращения количества признаков и упрощения задачи кластеризации 
принимается решение объединить датасеты с похожими типами сервисов. 

In [9]:
# Объединение данных о кафе и ресторанах
dining_data = pd.concat([cafe_data, gpd.read_file('data/source_data/restaurant.geojson')[['id', 'geometry']]])

In [10]:
# Объединение данных о торговых центрах и супермаркетах
shopping_data = pd.concat([gpd.read_file('data/source_data/supermarket.geojson')[['id', 'geometry']],
                          gpd.read_file('data/source_data/shopping_center.geojson')[['id', 'geometry']]])

In [11]:
# Объединение данных о детских садах и школах
edu_data = pd.concat([gpd.read_file('data/source_data/kindergarten.geojson')[['id', 'geometry']],
                          gpd.read_file('data/source_data/school.geojson')[['id', 'geometry']]])

In [12]:
# Объединение данных о поликлинниках и аптеках
health_data = pd.concat([gpd.read_file('data/source_data/pharmacy.geojson')[['id', 'geometry']],
                          gpd.read_file('data/source_data/polyclinic.geojson')[['id', 'geometry']]])

In [33]:
# Объединение данных о музеях и библиотеках
culture_data = pd.concat([gpd.read_file('data/source_data/museum.geojson')[['id', 'geometry']],
                          gpd.read_file('data/source_data/library.geojson')[['id', 'geometry']]])

In [14]:
# Все точечные сервисы объединяются в массив для дальнейшего анализа
services_points = [dining_data, shopping_data, edu_data, health_data, culture_data]

<b>1.4 Данные о зданиях</b>

In [18]:
buildings_data = gpd.read_file('data/source_data/building.geojson')
buildings_data.tail(3)

,building_id,address,building_date,building_area,living_area,storeys,population,central_heating,lift_count,central_hotwater,central_electricity,central_gas,refusechute,is_living,geometry
4259,122614,"г.Санкт-Петербург, набережная Макарова, дом 14...",1823,159.6,32.00,2.0,2.0,True,0.0,True,True,True,True,False,"MULTIPOLYGON (((30.28992 59.94590, 30.29008 59..."
4260,122615,"г.Санкт-Петербург, 7-я линия В.О., дом 6, лите...",1917,3398.8,1437.85,5.0,63.0,True,NaN,False,True,True,False,False,"MULTIPOLYGON (((30.28387 59.93718, 30.28476 59..."
4261,122616,"г.Санкт-Петербург, Средний проспект В.О., дом ...",2011,89327.9,46407.50,16.0,2900.0,True,20.0,True,True,False,False,True,"MULTIPOLYGON (((30.25126 59.93804, 30.25180 59..."


В задаче требуется анализировать расположение только жилых зданий в кварталах, поскольку в датасете building.geojson нет 
информации о функциональной принадлежности нежилых зданий (какие сервисы в них находятся). Поэтому требуется отфильтровать 
в датасете жилые здания. После первоначального ознакомления с датасетом можно предположить, что отфильтровать следует
по признаку is_living = True. Однако при этом можно наблюдать, что у зданий с признаком is_living = False могут присутствовать 
сведения о количестве жильцов в доме (population), а также о жилой площади (living_area).

In [28]:
buildings_data.shape

(4262, 15)

In [19]:
buildings_data[buildings_data.population.notnull()][['building_id', 'population', 'living_area', 'is_living']].head()

,building_id,population,living_area,is_living
27,32990,1122.0,17958.0,False
28,33114,1831.0,29294.3,False
29,33175,151.0,2422.5,True
31,33212,1702.0,27227.9,False
33,33294,427.0,6830.8,False


In [20]:
# В связи с этим жилые здания в датасете отфильтрованы с учетом трёх признаков: is_living, population, living_area
living_builds = buildings_data.loc[(buildings_data.population.notnull()) | (buildings_data.living_area.notnull()) | (buildings_data.is_living == True)]

In [25]:
# Дальнейший анализ показал, что в датасете с жилыми зданиями есть записи с дублирующимися идентификаторами зданий
living_builds[living_builds.duplicated(subset=['building_id'])][['building_id', 'population', 'living_area', 'is_living']].head(10)

,building_id,population,living_area,is_living
13,32353,NaN,NaN,True
14,32353,NaN,NaN,True
26,32627,NaN,NaN,True
37,33505,NaN,NaN,True
38,33505,NaN,NaN,True
39,33505,NaN,NaN,True
40,33505,NaN,NaN,True
41,33505,NaN,NaN,True
42,33505,NaN,NaN,True
64,33951,NaN,NaN,True


In [27]:
# Предполагается, что наличие дублирующихся идентификаторов зданий обусловлено некачественно выполненной выгрузкой исходных данных
# Дубликаты записей исключаются из датасета
living_builds = living_builds.drop_duplicates(subset = ['building_id'])
living_builds.shape

(1903, 15)

In [29]:
living_builds.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1903 entries, 10 to 4261
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   building_id          1903 non-null   int64   
 1   address              1305 non-null   object  
 2   building_date        1199 non-null   object  
 3   building_area        1199 non-null   float64 
 4   living_area          1199 non-null   float64 
 5   storeys              1231 non-null   float64 
 6   population           1199 non-null   float64 
 7   central_heating      1199 non-null   object  
 8   lift_count           513 non-null    float64 
 9   central_hotwater     1199 non-null   object  
 10  central_electricity  1199 non-null   object  
 11  central_gas          1199 non-null   object  
 12  refusechute          1199 non-null   object  
 13  is_living            1903 non-null   bool    
 14  geometry             1903 non-null   geometry
dtypes: bool(1), 

В датасете living_builds содержится довольно много информации о жилых зданиях. В частности, информация о количестве жителей зданий и количестве этажей, которую можно было бы использовать как характеристику густонаселенности квартала, в котором расположено здание. Однако в данных о количестве жителей и количестве этажей много пропусков (около 40%) и восполнить пропуски без искажения данных не представляется возможным. При этом известно, что Васильевский остров преимущественно застроен многоэтажными зданиями с минимальной малоэтажной застройкой. Поэтому по согласованию с заказчиком исследования для анализа населенности кварталов используются только сведения о геометрии жилых зданий (полигонах), прочие признаки из исследования исключаются.

In [30]:
living_builds = living_builds[['building_id', 'geometry']]

<b>1.5 Данные о прочих "полигональных" сервисах</b>

Исследовательский анализ показал (эта часть не включается в ноутбук для сокращения его размера), что оставшиеся датасеты содержат данные о сервисах с геометрией полигонов: znop.geojson (607 сервисов) и cemetery.geojson (4 сервиса). Предлагается для сокращения количества признаков объединить датасеты.

In [36]:
znop_data = pd.concat([gpd.read_file('data/source_data/cemetery.geojson')[['geometry']],
                          gpd.read_file('data/source_data/znop.geojson')[['geometry']]])

In [37]:
# Все полигональные сервисы объединяются в массив для дальнейшего анализа
services_polygons = [living_builds, znop_data]

На этом этапе исходные данные исследованы и обработаны. Получен датасет с данными о кварталах и массивы датасетов с данными точечных и полигональных сервисов. Далее на основании имеющихся данных будут сформированы признаки для дальнейшей кластеризации. 

<b>2 Формирование признаков</b>